In [0]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt


from tensorflow.examples.tutorials.mnist.input_data import read_data_sets as load_mnist
mnist = load_mnist("MNIST_data/", one_hot=True)

train_image = mnist.train.images.reshape(-1, 28, 28, 1)
test_image = mnist.test.images.reshape(-1, 28, 28, 1)

train_label = np.array(mnist.train.labels, dtype=np.bool)
test_label = np.array(mnist.test.labels, dtype=np.bool)

In [0]:
batch_size = 16
learning_rate = 0.001
hidden_num = 64
kernel_size = 3

log_dir = "./log/"
model_dir = "./model/"

In [0]:
X = tf.placeholder(shape=(None, 28, 28, 1), dtype=tf.float32) # 배치 개수는 정의되지 않아도 됩니다. 즉, 매 학습 스탭마다 배치 개수가 달라도 됩니다.
y = tf.placeholder(shape=(None, 10), dtype=tf.int32)

# Conv -> Pool -> Conv -> Pool -> Output
out = tf.layers.conv2d(inputs= X,
                       filters= hidden_num,
                       kernel_size= kernel_size,
                       activation=tf.nn.elu,
                       kernel_initializer=tf.random_normal_initializer())

out = tf.layers.max_pooling2d(inputs = out,
                              pool_size= 2,
                              strides = 2)

out = tf.layers.conv2d(inputs= out,
                       filters= hidden_num,
                       kernel_size= kernel_size,
                       activation=tf.nn.elu,
                       kernel_initializer=tf.random_normal_initializer())

out = tf.layers.max_pooling2d(inputs = out,
                              pool_size= 2,
                              strides = 2)

out = tf.layers.flatten(out)
out = tf.layers.dense(inputs= out, units = 10)

loss = tf.losses.softmax_cross_entropy(onehot_labels= y, logits= out)
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [7]:
#Tensorboard

!wget -nc https://raw.githubusercontent.com/mixuala/colab_utils/master/tboard.py
import tboard
ROOT=%pwd
url = tboard.launch_tensorboard(bin_dir=ROOT, log_dir=log_dir)

print(url)


summary_writer = tf.summary.FileWriter(log_dir)

summary_op = tf.summary.merge([
    tf.summary.scalar("Loss", loss)
])

File ‘tboard.py’ already there; not retrieving.

ngrok installed
status: tensorboard=True, ngrok=False
('tensorboard url=', u'http://0ff43c45.ngrok.io')


In [0]:
# Saver

saver = tf.train.Saver(max_to_keep=1000)

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fetch_dict = {"optim": optimizer, "loss": loss, "summ": summary_op}
    
    rand_idx = np.random.permutation(len(train_image))
    save_cnt = 0
    for step in range(0, len(rand_idx), batch_size):
        cur_idx = rand_idx[step:step+batch_size]
        result = sess.run(fetch_dict, feed_dict={X: train_image[cur_idx], y: train_label[cur_idx]})
        #Summary 입력!
        summary_writer.add_summary(result["summ"], global_step=step)
        if step%100 ==0:
            print(step, result["loss"])
            # 모델 저장!
            saver.save(sess=sess, save_path=model_dir+"%d.ckpt"%save_cnt)
            save_cnt +=1

In [0]:


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    l = sess.run(loss, feed_dict = {X: train_image[0:10], y: train_label[0:10]})
    print('모델 불러오기 전 에러 값:'%l)
    
    # 모델 불러오기
    saver.restore(sess=sess, save_path=model_dir+"10.ckpt")
    
    
    l = sess.run(loss, feed_dict = {X: train_image[0:10], y: train_label[0:10]})
    print('모델 불러온 후 에러 값: %f'%l)

In [0]:
# Tensorboard 예시2
import tboard
ROOT=%pwd
url = tboard.launch_tensorboard(bin_dir=ROOT, log_dir="/tmp/tensorflow/mnist/logs/mnist_with_summaries/")
while True:
    print(url)
# !wget -nc https://raw.githubusercontent.com/tensorflow/tensorflow/r1.9/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py
# !python mnist_with_summaries.py

